Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.




## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [25]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_caixabinaria as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_simples as funcao_cruzamento
from funcoes import mutacao_caixabinaria as funcao_mutacao

## Códigos e discussão



In [26]:
#CONSTANTES

#Constantes relacionadas à busca:
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

#Constantes relacionadas ao problema a ser resolvido:
LIMITE_DE_PESO = 15
OBJETOS = {
    #Dicionário contendo os objetos com seu peso (podemos pensar em kg) e valor (podemos pensar em $):

    "Vinil do Turn Off The Lights I - Kim Petras": {
        "peso": 2,
        "valor": 2500,
    },
    "Livro do Harry Potter escrito pela Hermione Granger": {
        "peso": 3,
        "valor": 1500,
    },
    "Fragmento de espelho que eu quebrei de manha quando me olhei após recém acordar": {
        "peso": 3,
        "valor": 7000,
    },
    "Camiseta preta escrito não gosto de cores neutras": {
        "peso": 3,
        "valor": 150,
    },
    "Fone de ouvido sem fio": {
        "peso": 0.6,
        "valor": 2400,
    },
    "Fio para o fone de ouvido sem fio": {
        "peso": 3.5,
        "valor": 3000,
    },
    "Capa da invisibilidade reversa - te deixa visível": {
        "peso": 16,
        "valor": 1000,
    },
    "Um fardo de água com gás que caiu de um caminhão": {
        "peso": 8,
        "valor": 5000,
    },
    "Pílulas de dopaminas para alunos da Ilum": {
        "peso": 2,
        "valor": 1500,
    },
    "Bolo de aniversário estilo anos 2000 com desenho da Barbie": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [27]:
#Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [30]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total}kg de sal (igual na idade média) "
    f"e peso de {peso_total} sacos de batata."
)

Maior valor: 16400 | Peso: 12.1
Maior valor: 17900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Fio para o fone de ouvido sem fio
+ Fone de ouvido sem fio
+ Fragmento de espelho que eu quebrei de manha quando me olhei após recém acordar
+ Livro do Harry Potter escrito pela Hermione Granger
+ Pílulas de dopaminas para alunos da Ilum
+ Vinil do Turn Off The Lights I - Kim Petras

Com isso, sua mochila terá o valor de 17900kg de sal (igual na idade média) e peso de 14.1 sacos de batata.


## Conclusão


Nesse experimento, utilizamos de um problema de lógica simples que consiste na ideia de "quantos objetos com valor x até um peso máximo y você consegue carregar para que seja o maior preço possível?". Existe a possibilidade de resolver por de um modo mais tradicional e de um modo com um Algoritmo Genético, sendo esse último o foco aqui. Esse problema foi utilizado para que pudessemos entender como aplicar uma restrição.
A restrição é aplicada para, por exemplo aqui, retringir o limite de peso da mochila. Desse modo, dentro desse limite, os itens mais caros seriam escolhidos. E nosso problema, por consequência, é um problema de maximização.
Seguimos uma lógica baseada no problema das caixas binárias onde a mochila (indivíduo) pode assumir alguns determinados valores binários de gene (itens - 1 se estão dentro do limite; 0 se não se adequam ao limite de peso da mochila), e dessa forma podemos realizar as operações de seleção, cruzamento e mutação, que são típicas de um Algoritmo Genético. Dentro disso, ainda, devemos levar em conta o preço dos itens. Podemos pensar que enquanto o algoritmo maximiza o preço, aplica restrição no peso, como se fosse uma minimização.



## Playground

